In [1]:
import json
import numpy as np

In [2]:
class MMPositions():
    '''
    '''
    
    def __init__(self, x_stage_name, y_stage_name):
        self.x_stage_name = x_stage_name
        self.y_stage_name = y_stage_name
        self.positions = []

    def _array(self, type, value=[]):
        return  {
            "type": type,
            "array": value,
        }
            
    def _scalar(self, type, value=""):
        return {
            "type": type,
            "scalar": value
        }

    def _format_position(self, x, y, label):
        fmt_pos = {
            "DefaultXYStage": self._scalar("STRING"),
            "DefaultZStage": self._scalar("STRING"),
            
            "DevicePositions": self._array("PROPERTY_MAP", [
                {
                    "Device": self._scalar("STRING", self.y_stage_name),
                    "Position_um": self._array("DOUBLE", [y])
                },
                {
                    "Device": self._scalar("STRING", self.x_stage_name),
                    "Position_um": self._array("DOUBLE", [x])
                }
            ]),
            "GridCol": self._scalar("INTEGER", 0),
            "GridRow": self._scalar("INTEGER", 0),
            "Label": self._scalar("STRING", label),
            "Properties": self._scalar("PROPERTY_MAP", {})
        }
        
        return fmt_pos


    def add(self, x, y):
        self.positions.append((x, y))
    
    def to_dict(self):
        # Get all positions formatted
        stage_positions = [self._format_position(x, y, f"Pos{i}") for i, (x, y) in enumerate(self.positions)]

        # Append to base data structure
        data = {
            "encoding": "UTF-8",
            "format": "Micro-Manager Property Map",
            "major_version": 2,
            "minor_version": 0,
            "map": {
                "StagePositions": self._array("PROPERTY_MAP", stage_positions)
            }
        }
        
        return data

In [12]:
angle = np.radians(0)

center_x = 37730
center_y = 28110
w = 10_000
step = 100

# Number of steps required to translate a square across an angled illumination line

if angle < np.radians(90):
    n_steps = 1 + abs(round((w / step) * np.sqrt(2) * np.sin(angle + np.pi/4)))
else:
    n_steps = 1 + abs(round((w / step) * np.sqrt(2) * np.sin(2*np.pi - angle + np.pi/4)))


dx = step * np.cos(angle)
dy = step * np.sin(angle)

xs = range(n_steps) * dx
xs -= np.mean(xs)
xs += center_x

ys = range(n_steps) * dy
ys -= np.mean(ys)
ys += center_y

print(n_steps)

101


In [13]:
mmpos = MMPositions(x_stage_name="TDC001-27252765-X", y_stage_name="TDC001-27504502-Y")

for (x, y) in zip(xs, ys):
    mmpos.add(x, y)

with open('./positions.pos', 'w') as json_file:
    json.dump(mmpos.to_dict(), json_file, indent=4)